In [1]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE


In [2]:
train_df = pd.read_csv('train_data2.csv')

In [3]:
test_df = pd.read_csv('test_data2.csv')

In [4]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [5]:
# Tokenize sentences into words
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in train_df['tweet'].astype(str)]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save or load the model
w2v_model.save("word2vec_model2.model")
# w2v_model = Word2Vec.load("word2vec_model.model")

# Get the word vectors
word_vectors = w2v_model.wv

In [6]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(str(sentence)) for sentence in train_df['tweet']])
X_test = np.array([vectorize(str(sentence)) for sentence in test_df['tweet']])

In [7]:
Y_train = train_df['decision']

In [8]:
Y_test = test_df['decision']

In [9]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial')
clf.fit(X_train, Y_train)

/Users/chandrikamukherjee/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(Y_test, Y_pred))

Accuracy: 0.6548579847753131


In [11]:
precision = precision_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
print(precision, f1, recall)

0.6350143097094522 0.6287080690794743 0.6548579847753131


In [12]:
from sklearn.metrics import classification_report

In [13]:
report = classification_report(Y_test, Y_pred, target_names=['negative','neutral','positive'])

In [14]:
print(report)

              precision    recall  f1-score   support

    negative       0.52      0.20      0.29      9778
     neutral       0.59      0.53      0.56     18023
    positive       0.69      0.85      0.76     33284

    accuracy                           0.65     61085
   macro avg       0.60      0.53      0.54     61085
weighted avg       0.64      0.65      0.63     61085

